In [1]:
import pandas as pd
import requests
from requests_negotiate_sspi import HttpNegotiateAuth
from pandas.io.json import json_normalize
import plotly.express as px

below is a call to the enetities and getting every single dbxmarker. All we see in this view is the guid, the name, and the entity URL 

In [2]:
url = r'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker?page=0&entitiesPerPage=50000'
r = requests.get(url,auth=HttpNegotiateAuth())
r.json()

{'pageNumber': 0,
 'entityType': 'DbxMarker',
 'entityTypeName': 'Well top',
 'entityCount': 40934,
 'entitiesPerPage': 50000,
 'pageCount': 1,
 'firstPageURL': 'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker?page=0&entitiesPerPage=50000',
 'previousPageURL': None,
 'nextPageURL': None,
 'lastPageURL': 'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker?page=0&entitiesPerPage=50000',
 'entityList': [{'guid': '00025B92EA4E4D5B9658FEDB83AD5A9E',
   'name': '18-3_TS - ODOPTU OP-14',
   'entityURL': 'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker/00025B92EA4E4D5B9658FEDB83AD5A9E'},
  {'guid': '000433A3A9CA410FA6FA4CBE456CA548',
   'name': '12 Top - 206ST1',
   'entityURL': 'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker/000433A3A9CA410FA6FA4CBE456CA548'},
  {'guid': '0005656C1E72480BAF542941E577284E',
   'name': '12 Top - 204ST1',
   'entityURL': 'http://hoewap607:50231/api/s

In [3]:
df = pd.json_normalize(r.json()['entityList'])
df.iloc[888,:]

guid                          055FBD76B56F4832A2DDEEA19621B125
name                                       13-3/8" - ODOPTU_11
entityURL    http://hoewap607:50231/api/studio/petrel/entit...
Name: 888, dtype: object

### Object data 

First we will look at object data for the entire marker collection, then we will look at the obnject data for a particular marker

In [105]:
url = 'http://hoewap607:50231/api/studio/petrel/objects/P_PET_SAKP/SAK01/DbxMarkerCollection/355D5F973FA94203A8329313232C8036'
well_tops_main_query = requests.get(url,auth=HttpNegotiateAuth())


In [126]:
well_tops_main_query.json()['properties']

{'Id': 21272445,
 'Guid': '355D5F973FA94203A8329313232C8036',
 'Name': 'EMSDB_AD',
 'JsonDoc': None,
 'JsonAdminDoc': None,
 'Comments': 'Well tops for use in EMSDB',
 'SemName': 'EMSDB_AD',
 'SemRemarks': 'Well tops for use in EMSDB',
 'SRD': 0.0,
 'SRDName': 'SRD',
 'SRDVelocity': '{"Replacement_Velocity":"1480"}',
 'SRDTimeShift': None,
 'TimeVerticalReference': {'entityType': 'DbxCoordinateSystem',
  'uniqueId': '4629E8B9-8543-58BA-AB75-EAD09C8D60B2'},
 'ParentCollection': {'entityType': 'DbxCollection',
  'uniqueId': '70E1491B52654D8594788E26AFF26226'},
 'CustomDomainObjects': [],
 'Stratigraphies': [{'entityType': 'DbxStratigraphy',
   'uniqueId': '355D5F973FA94203A8329313232C8036'}],
 'MarkerFaults': [],
 'MarkerHorizons': [{'entityType': 'DbxMarkerHorizon',
   'uniqueId': '3BBC86D5C6AC4A959F330C6CD580AB92'},
  {'entityType': 'DbxMarkerHorizon',
   'uniqueId': '404E20B4C34441D08D2B4A6DB23C96F5'},
  {'entityType': 'DbxMarkerHorizon',
   'uniqueId': '9F2249DAF0EE4D30A736666ADB56A6

In [123]:
[x for x  in well_tops_main_query.json()]

['properties', 'propertyDisplayNames']

In [121]:
%%time
url = 'http://hoewap607:50231/api/studio/petrel/objects/P_PET_SAKP/SAK01/DbxMarker/'

#blank df to populate with data
marker_df = pd.DataFrame()
prop_value_df = pd.DataFrame()
unique_id_list = pd.DataFrame([x for x in well_tops_main_query.json()['properties']['Markers']]).uniqueId.values # r.json()['propertyDisplayNames']
for x in unique_id_list:
    id_url = url + x
    r = requests.get(id_url,auth=HttpNegotiateAuth())
    json_obj = r.json()
    zone = json_obj['properties']['MarkerSurface']
    well_name = json_obj['properties']['MarkerBorehole']
    uwi = json_obj['properties']['UWI']
    x = json_obj['properties']['MarkerPosition']['Geometry']['x']
    y = json_obj['properties']['MarkerPosition']['Geometry']['y']
    z = json_obj['properties']['MarkerPosition']['Geometry']['z']
    marker_guid = json_obj['properties']['Guid']
    marker_dict = {
        'zone':zone,
        'well_name':well_name,
        'uwi':uwi,
        'x':x,
        'y':y,
        'z':z,
        'marker_guid':marker_guid}
    marker_df = marker_df.append(marker_dict,ignore_index=True)
    #this will grab all of the property values for each marker
    for x in json_obj['properties']['MarkerPropertyValues']:
        value_Guid = x['Guid']
        StringValue = x['StringValue']
        FloatValue = x['FloatValue']
        FloatValueUom = x['FloatValueUom']
        IntegerValue = x['IntegerValue']
        IntegerValueUom = x['IntegerValueUom']
        value_id = x['Id']
        row_add_dict = {
            'marker_guid':marker_guid,
            'value_id':value_id,
            'value_Guid':value_Guid,
            'StringValue':StringValue,
            'FloatValue':FloatValue,
            'FloatValueUom':FloatValueUom,
            'IntegerValue':IntegerValue,
            'IntegerValueUom':IntegerValueUom,
                       }
        prop_value_df = prop_value_df.append(row_add_dict,ignore_index=True)
        
print('finished')




finished
Wall time: 58min 21s


In [120]:
marker_df.head()

,marker_guid,uwi,well_name,x,y,z,zone
0,580DE250E9DC47E7B62CBE5F61C91BAE,548250188600,ARKUTUN-DAGI DP-18U,143.689885,52.408744,-1784.710389,TD
1,BA3C1883584B43D9934C644FF369B08E,548250188600,ARKUTUN-DAGI DP-18U,143.683551,52.437301,-1738.713411,11-1_MFS (Zone_XI)
2,2D9CAD14891F408D87A5925527E6A773,548250188600,ARKUTUN-DAGI DP-18U,143.682488,52.438742,-1715.799440,10-2_PSB (Zone X-2)
3,09CB9E18182649FB9C1C7A061B3D9E96,548250188600,ARKUTUN-DAGI DP-18U,143.680850,52.440404,-1681.730853,10-1_PSB (Zone_X-1)
4,6A76B5D086C94946BD0ECDB052AC4DFF,548250189300,ARKUTUN-DAGI DP-26L,143.682921,52.400499,-2066.370539,TD


This list shows aaaallll the different types of properties associated with this individual pick

In [5]:
url = 'http://hoewap607:50231/api/studio/petrel/objects/P_PET_SAKP/SAK01/DbxMarkerPropertyType/49BE39E4FDCE4A55A018E315DED29246'
r = requests.get(url,auth=HttpNegotiateAuth())
r.json()

{'properties': {'Id': 21272704,
  'Guid': '49BE39E4FDCE4A55A018E315DED29246',
  'CreateDate': '2021-04-05T13:42:44',
  'CreateUser': 'NJKARSK',
  'UpdateDate': '2021-04-05T13:42:44',
  'UpdateUser': 'NJKARSK',
  'JsonDoc': None,
  'JsonAdminDoc': None,
  'Name': 'Thickness_of_NetSand_MD',
  'Comments': None,
  'DataType': 'Float',
  'RemoteCreateDate': '2021-04-05T13:42:44',
  'RemoteCreateUser': 'NJKARSK',
  'RemoteUpdateDate': '2021-04-05T13:42:44',
  'RemoteUpdateUser': 'NJKARSK',
  'PropertyType': 'ZoneProperty',
  'MarkerCollection': {'entityType': 'DbxMarkerCollection',
   'uniqueId': '355D5F973FA94203A8329313232C8036'},
  'ParentTemplate': {'entityType': 'DbxTemplate',
   'uniqueId': '0000001D000000000000000000000000'},
  'MarkerPropertyValues': [{'entityType': 'DbxMarkerPropertyValue',
    'uniqueId': '8A9FE5D2783C4E6F9EE64823140FC19B|49BE39E4FDCE4A55A018E315DED29246'},
   {'entityType': 'DbxMarkerPropertyValue',
    'uniqueId': '0D1A70C67B0A4130956D2FD06AC2A4B0|49BE39E4FDCE4A5

In [6]:
url = 'http://hoewap607:50231/api/studio/petrel/objects/P_PET_SAKP/SAK01/DbxMarkerPropertyType/49BE39E4FDCE4A55A018E315DED29246'
r = requests.get(url,auth=HttpNegotiateAuth())
r.json()

{'properties': {'Id': 21272704,
  'Guid': '49BE39E4FDCE4A55A018E315DED29246',
  'CreateDate': '2021-04-05T13:42:44',
  'CreateUser': 'NJKARSK',
  'UpdateDate': '2021-04-05T13:42:44',
  'UpdateUser': 'NJKARSK',
  'JsonDoc': None,
  'JsonAdminDoc': None,
  'Name': 'Thickness_of_NetSand_MD',
  'Comments': None,
  'DataType': 'Float',
  'RemoteCreateDate': '2021-04-05T13:42:44',
  'RemoteCreateUser': 'NJKARSK',
  'RemoteUpdateDate': '2021-04-05T13:42:44',
  'RemoteUpdateUser': 'NJKARSK',
  'PropertyType': 'ZoneProperty',
  'MarkerCollection': {'entityType': 'DbxMarkerCollection',
   'uniqueId': '355D5F973FA94203A8329313232C8036'},
  'ParentTemplate': {'entityType': 'DbxTemplate',
   'uniqueId': '0000001D000000000000000000000000'},
  'MarkerPropertyValues': [{'entityType': 'DbxMarkerPropertyValue',
    'uniqueId': '8A9FE5D2783C4E6F9EE64823140FC19B|49BE39E4FDCE4A55A018E315DED29246'},
   {'entityType': 'DbxMarkerPropertyValue',
    'uniqueId': '0D1A70C67B0A4130956D2FD06AC2A4B0|49BE39E4FDCE4A5

Lets make a dictionary of the different  DbcMarkerPropertyValues so we can associate the ID with the actual value

In [128]:
url = r'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker?page=10&entitiesPerPage=50000'
r = requests.get(url,auth=HttpNegotiateAuth())
r.json()

{'pageNumber': 10,
 'entityType': 'DbxMarker',
 'entityTypeName': 'Well top',
 'entityCount': 40869,
 'entitiesPerPage': 50000,
 'pageCount': 1,
 'firstPageURL': 'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker?page=0&entitiesPerPage=50000',
 'previousPageURL': 'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker?page=9&entitiesPerPage=50000',
 'nextPageURL': None,
 'lastPageURL': 'http://hoewap607:50231/api/studio/petrel/entities/P_PET_SAKP/SAK01/DbxMarker?page=0&entitiesPerPage=50000',
 'entityList': []}

In [19]:
df = pd.json_normalize(r.json()['entityList'])

In [20]:
df

,guid,name,entityURL
0,00025B92EA4E4D5B9658FEDB83AD5A9E,18-3_TS - ODOPTU OP-14,http://hoewap607:50231/api/studio/petrel/entit...
1,000433A3A9CA410FA6FA4CBE456CA548,12 Top - 206ST1,http://hoewap607:50231/api/studio/petrel/entit...
2,0005656C1E72480BAF542941E577284E,12 Top - 204ST1,http://hoewap607:50231/api/studio/petrel/entit...
3,0005C747A90C491A9EDF1849C2698201,2a-0 bse - 227,http://hoewap607:50231/api/studio/petrel/entit...
4,000623E21C424588845B2271E142AC69,2b Base - ODOPTU ODWI-01 PILOT,http://hoewap607:50231/api/studio/petrel/entit...
...,...,...,...
40864,FFF7B18BD13041208F920B670FF1BC8B,13-2_PSB - DAGI_06,http://hoewap607:50231/api/studio/petrel/entit...
40865,FFF7DE4AD5BA49C6A56C53CE49344405,17structural_axis - CHAYVO NC-1 PA,http://hoewap607:50231/api/studio/petrel/entit...
40866,FFF9810474224CDB95FE15071378EDA2,Aq2a_FS_4 - ODOPTU_10,http://hoewap607:50231/api/studio/petrel/entit...
40867,FFFE007AC02A438E8A40CC19B42FBDBB,16cl25 - CHAYVO O-26S,http://hoewap607:50231/api/studio/petrel/entit...
